In [240]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as functional
from torch.utils.data import Dataset, DataLoader
from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np


In [241]:
data_dir = "/Users/Tad/Documents/faceoffs"
data_initial = pd.read_csv("training_data_all_offensive_offensive.csv")

In [242]:
data_no_na = data_initial.dropna() # data should already have no NAs due to numerical imputation
data = data_no_na.select_dtypes(['number'])
x = data.loc[:, data.columns != 'net_xg']
x = x.loc[:, x.columns != 'net_xg']
y = data['net_xg']

X_train, X_intermediate, y_train, y_intermediate = train_test_split(x, y, train_size = 0.65, test_size=0.35, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_intermediate, y_intermediate, test_size=0.571, random_state=42)

# Above two lines in unison accomplish a 65-15-20 split for train-test-validation

In [243]:
device_selection = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_selection)
print(device_selection)
print(device)

if torch.cuda.is_available():
        torch.cuda.empty_cache()

train_batch_size = 64
val_batch_size = 64
test_batch_size = 64

cpu
cpu


In [244]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        x = torch.tensor(self.df.iloc[idx, :-1].values, dtype=torch.float32)
        y = torch.tensor(self.df.iloc[idx, -1:].values, dtype=torch.float32)
        return x, y

train_df =  pd.concat([X_train, y_train], axis=1)  # pandas DataFrame containing training data
valid_df = pd.concat([X_val, y_val], axis=1) # pandas DataFrame containing validation data
test_df = pd.concat([X_test, y_test], axis=1)  # pandas DataFrame containing test data
train_dataset = CustomDataset(train_df)
valid_dataset = CustomDataset(valid_df)
test_dataset = CustomDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=val_batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

# Credit to ChatGPT for generating this cell of code in response to the following prompt: design a dataloader that accepts the output of a pandas train-validation-test split. Basically write code for a PyTorch dataloader that accepts pandas dataframes as arguments

In [245]:
class VanillaNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_rate):
        super(VanillaNeuralNet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout_rate = dropout_rate
        
        self.fc1 = nn.Linear(self.input_size, self.hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=self.dropout_rate)

        self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc3 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc4 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc5 = nn.Linear(self.hidden_size, self.hidden_size)

        self.fc6 = nn.Linear(self.hidden_size, 1)

    def forward(self, x):

        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc4(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc5(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc6(x)
        
        return x


In [247]:
import optuna

num_epochs = 3
trial_count = 1

learning_rate = 0.01
loss_func = nn.MSELoss()

def train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device):
    training_losses = np.array([])
    validation_losses = np.array([])
    for epoch in range(num_epochs):
        print("\nEPOCH " +str(epoch+1)+" of "+str(num_epochs)+"\n")
        ########################### Training #####################################
        model.train(True)
        train_loss = 0
        count = 0
        for batch, (X, y) in enumerate(train_loader):
            X = X.float().to(device)
            y = y.float().to(device)
            optimizer.zero_grad()
            prediction = model(X).to(device)
            loss = loss_func(prediction, y.view(-1, 1))
            loss.backward()
            optimizer.step()
            scheduler.step()
            train_loss += loss.float().item()
            count += 1
        training_losses = np.append(training_losses, train_loss / count)
        print("Training loss:", train_loss / count)
        model.train(False)

        ########################### Validation #####################################
        model.eval()
        val_loss = 0
        count = 0
        with torch.no_grad():
            for (X, y) in valid_loader:
                X = X.to(device)
                y = y.to(device)
                prediction = model(X).to(device)
                loss = loss_func(prediction, y.view(-1, 1))
                val_loss += loss.float().item()
                count += 1
        validation_losses = np.append(validation_losses, val_loss / count)
        print("Validation loss:", val_loss / count)
    return validation_losses[-1]

def objective(trial):
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    hidden_size = trial.suggest_int("hidden_size", 10, 100)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.9)

    model = VanillaNeuralNet(input_dim, hidden_size, dropout_rate).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    return train(model, train_loader, valid_loader, loss_func, optimizer, scheduler, num_epochs, device)

study = optuna.create_study(direction="minimize")
input_dim = 529
output_dim = 1
hidden_dim = 300
dropout = 0.1
model = VanillaNeuralNet(input_dim, hidden_dim, dropout)
study.optimize(objective, n_trials=trial_count)
print(study.best_params)

[I 2023-04-24 00:58:01,985] A new study created in memory with name: no-name-ae2e21f5-f7d9-41d6-adc3-f97a1cb00fe9



EPOCH 1 of 3

Training loss: 0.007663921196550383
Validation loss: 0.006928937821520851

EPOCH 2 of 3

Training loss: 0.007660804611879354
Validation loss: 0.006928937821520851

EPOCH 3 of 3

Training loss: 0.007672729907888856


[I 2023-04-24 01:01:04,926] Trial 0 finished with value: 0.006928937821520851 and parameters: {'lr': 0.0001568731302967673, 'weight_decay': 7.238297157653941e-05, 'hidden_size': 63, 'dropout_rate': 0.38540270713833225}. Best is trial 0 with value: 0.006928937821520851.


Validation loss: 0.006928937821520851
{'lr': 0.0001568731302967673, 'weight_decay': 7.238297157653941e-05, 'hidden_size': 63, 'dropout_rate': 0.38540270713833225}


In [248]:
best_params = study.best_trial.params
print(best_params)
model = VanillaNeuralNet(input_dim, best_params["hidden_size"], best_params["dropout_rate"])
optimizer = optim.Adam(model.parameters(), lr=best_params["lr"], weight_decay=best_params["weight_decay"])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_func = nn.MSELoss()

# Train the model with the best hyperparameters on the entire dataset
for epoch in range(num_epochs):
    ########################### Training #####################################
    model.train(True)
    train_loss = 0
    count = 0
    for batch, (X, y) in enumerate(train_loader):
        X = X.float().to(device)
        y = y.float().to(device)
        prediction = model(X).to(device)
        optimizer.zero_grad()
        loss = loss_func(prediction, y.unsqueeze(1))
        loss.backward()
        optimizer.step()
        train_loss += loss.float().item()
        count += 1
    scheduler.step()
    print("Epoch", epoch+1, "Training loss:", train_loss / count)

# Save the best model
torch.save(model.state_dict(), "faceoffs_best_model_params.pt")
# save the trained model
torch.save(model, "faceoffs_best_model.pt")

{'lr': 0.0001568731302967673, 'weight_decay': 7.238297157653941e-05, 'hidden_size': 63, 'dropout_rate': 0.38540270713833225}


C:\Users\Tad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([35, 1, 1])) that is different to the input size (torch.Size([35, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1 Training loss: 0.0019167028281965215
Epoch 2 Training loss: 0.0017039389753705817
Epoch 3 Training loss: 0.001693652417335385


In [253]:
test_loss = 0
y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device)
        prediction = model(X).to(device)
        # Remove extra dimensions from y
        y = y.unsqueeze(1).squeeze()
        loss = loss_func(prediction.squeeze(), y)
        test_loss += loss.float().item()
        y_true.extend(y.cpu().numpy())
        y_pred.extend(prediction.cpu().numpy().squeeze())

test_loss /= len(test_loader)
print(f"Test Loss: {test_loss:.4f}")

from sklearn.metrics import r2_score
r2 = r2_score(y_true, y_pred)
print("True labels:", y_true)
print("Predictions:", y_pred)
print(f"R-squared Coefficient: {r2:.4f}")


Test Loss: 0.0017
True labels: [0.00961186, 0.00920247, 0.0, 0.0670891, 0.0, 0.00920247, 0.00920247, 0.011285, 0.00920247, 0.0, 0.00920247, 0.00920247, 0.00920247, 0.021146, 0.00920247, 0.00920247, 0.00920247, 0.130848, 0.00920247, 0.0, 0.00920247, 0.0241725, 0.00920247, 0.00920247, 0.00920247, 0.00920247, 0.0, 0.00920247, 0.00920247, 0.00960776, 0.0152522, 0.0286671, 0.00920247, 0.00920247, 0.00920247, 0.0538394, 0.00920247, 0.0134791, 0.0, 0.00920247, 0.0326136, 0.00920247, 0.00920247, 0.0105988, 0.0, 0.00920247, 0.00920247, 0.0117993, 0.0158291, 0.0119275, 0.00920247, 0.0918472, 0.018819, 0.0106601, 0.00920247, 0.0234173, 0.0105942, 0.0, 0.0, 0.0458232, 0.00920247, 0.00920247, 0.00920247, 0.0305091, 0.00920247, 0.0398371, 0.0271143, 0.00920247, 0.00920247, 0.00920247, 0.00920247, 0.0, 0.00920247, 0.0, 0.033239, 0.00920247, 0.00920247, 0.00920247, 0.00920247, 0.00920247, 0.0271564, 0.121453, 0.0186784, 0.00920247, 0.00920247, 0.0130433, 0.00920247, 0.00920247, 0.00920247, 0.00920247,